<a href="https://colab.research.google.com/github/hadeelbkh/GeminiAPI/blob/master/Gemini_Final_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [4]:
import time
from tqdm import tqdm
import pathlib
import google.generativeai as genai
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
from PIL import Image
import pytesseract
from IPython.display import Markdown

In [5]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

# Gemini Final Exercise

You're an astronomy student who's very curious about the Apollo 11 missions,
and through your research, you've found a lot of different types of data (otherwise known as multimodal) from NASA's
public archive.

1. Text: You have the full final NASA report post-mission, spanning over 300
pages of incredibly informative content that details a summary of everything
that happened as well as conclusions that NASA researchers and engineers
came to. For the sake of this exercise, we've selected 3 particularly interesting pages, and converted them to images (you'll see soon why).

2. Video: You also have several clips of the famous Neil Armstrong and Buzz Aldrin footage as they
first stepped onto the moon, containing highlights of their moonwalks as well
as raising the American flag.

3. Audio: Finally, you have highlights from the audio recorded throughout the
mission, which provides insights into how communication between the astronauts
occurred as well as from the astronauts to mission control.

Now, you want to search through and summarize this information for your
upcoming research paper. Using your newfound skills from this course, you
can accomplish this using Gemini! In particular, we will build a Retrieval Augmented Generation (RAG) system that you can directly interact with.

## Data Preparation

Before we begin, ensure that you've uploaded the resources.zip folder and unzipped it using the following command:

In [6]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-09-10 07:17:54--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.141.72, 104.19.139.72, 104.19.138.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.141.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M   206MB/s    in 1.3s    

2024-09-10 07:17:56 (206 MB/s) - ‘resources.zip’ saved [286142532/286142532]



In [7]:
!unzip resources.zip

Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl


As we saw throughout this course, when working with different types of data,
we first need to parse it in a way that Gemini can understand. We will prepare our data by extracting all file names from the `resources` directory.

In [8]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [9]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/video/OneSmallStepCompilation.mov
resources/video/RaisingTheAmericanFlag.mov
resources/video/Apollo11MoonwalkMontage.mov
resources/video/Apollo11Intro.mov
resources/video/BuzzDescendsCompilation.mov
resources/video/Apollo11PlaqueComparison.mov
resources/text/images-333.jpg
resources/text/images-023.jpg
resources/text/images-020.jpg
14


You should expect to see 14 files.

## Retrieval Augmented Generation (RAG)

To showcase how we build a RAG, we will first build one for the Text case, and generalize it further after. Here is the general idea:
1. **Data Preparation** (done above): We first collected various types of data from NASA's public archive related to the Apollo 11 mission, including text, video, and audio files.
2. **Data Extraction and Summarization**: Extract the multimodal data from images, e.g. extract text from images using Optical Character Recognition (OCR), and use Gemini to generate summaries using a specialized prompt.
3. **Embedding Generation**: Convert the generated summaries into vector embeddings using Gemini's Text Embedding Model. These embeddings represent the summaries in a numerical format suitable for efficient similarity searches.
4. **Creating a Vector Database**: A Vector database was created to store the embeddings. This database facilitates fast and efficient retrieval of relevant documents based on similarity searches. We chose to use Chroma DB.
5. **Querying the RAG System**: For a given query, the system retrieves the most relevant documents (based on their embeddings) and generates a response using the retrieved documents as context.

Something important to note is that RAGs are usually used only when there is a surplus of data. In other words, if the data can't fit into the model prompt. In this case, the data we provided likely can fit into Gemini's 1 million token window, but for the sake of simplicity and restrictions of Google Colab's runtime, we opted to use a smaller set of data.

### Text

We will use Tesseract OCR (Optical Character Recognition) to extract text from images of the NASA report.

In [10]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

Let's create a function to take in our images of a PDF, transcribe them into text, and summarize each of them.

In [11]:
def create_text_summary():
  path = pathlib.Path("resources/text")

  text_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

  images = []
  text_summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue

    image = Image.open(f)
    # TODO: what inputs would you pass to the Gemini model?
    response = model.generate_content([text_summary_prompt, pytesseract.image_to_string(image)])

    images.append(image) # TODO
    text_summaries.append(response.text) # TODO

  return images, text_summaries

In [12]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)

In [13]:
image_files, text_summaries = create_text_summary()

Now, we can check out the generated summaries of the three pages we have!

In [14]:
for text_summary in text_summaries:
  print(text_summary)

This document outlines ground rules and assumptions for an EPS analysis.  It covers battery usage, S-band equipment operation, rendezvous radar, PGNCS operation, and forward window heater status during a lunar mission. 

This document outlines the Apollo 11 mission timeline, focusing on the launch and translunar coast phases. It details the launch time, orbit insertion, and major events during the 73 hours of translunar coast, including TLI, CSM/LM transposition, SIVB separation, propulsive venting, and midcourse corrections. 

This document outlines the flight plan for Apollo 11, a G-type mission lunar landing. The plan includes details on the launch date, trajectory, and crew activities. Changes to the flight plan require approval from the Crew Procedures Control Board and should be submitted through a Crew Procedures Change Request. The document is under configuration control and any requests for changes or additional copies should be directed to Mr. W. J. North, Chief, Flight Crew 

We create the Chroma database using the generated summaries. You might be wondering what Vector DB and Chroma DB are.

**Vector Database**: A specialized database designed to store and manage high-dimensional vectors, which are numerical representations of data points. It allows efficient similarity searches to find vectors (and their corresponding data) that are close to a given query vector.

**Chroma DB**: An implementation of a vector database used to store and retrieve vector embeddings. These embeddings are generated from our summaries and allow us to perform efficient similarity searches.


In [15]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input, # TODO: What would we pass in as content?
                                task_type="retrieval_document", # TODO: What sort of task_type would this be? Check out the Gemini docs for what options there are.
                                title=title)["embedding"]

In [16]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction()) # TODO: Create a chroma db using the name and above embedding function. Hint: check out `get_or_create_collection`

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [17]:
text_db = create_chroma_db(text_summaries, "Apollo11_DB") # TODO: Create a db for all the text summaries using the above function

Let's also take a peak at the `text_db` and ensure that embeddings were generated:

In [18]:
data = {
    'embeddings': text_db.peek()['embeddings'],
    'documents': text_db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.08461565524339676, -0.00042855812353082, -0...",This document outlines ground rules and assump...
1,"[0.07635721564292908, -0.025517527014017105, 0...",This document outlines the Apollo 11 mission t...
2,"[0.0762612447142601, 0.013945112936198711, 0.0...",This document outlines the flight plan for Apo...


You should see a column called `embeddings` with what are seemingly random values, but these values are actually high-dimensional vectors that represent the semantic meaning of your summaries.

Now let's actually try querying our information. We'll test a simple example like getting some file that has to do with the Apollo 11 Flight Plan.

In [19]:
def get_relevant_files(query, db):
  results = db.query(query_texts=[query], n_results=3) # TODO: query from the db to get the top few results
  return results["ids"][0] # TODO

In [20]:
files = get_relevant_files("Apollo 11 Flight Plan", text_db)
print(files)

['2', '1', '0']


You should expect to see something like `['1', '0', '2']`. This means that the first entry in the `text_db` is most similar. If you look above at our `pd.DataFrame` output, the document with id 1 is the document about the Apollo 11 Flight Plan, so this is working as we expected!

### Video and Audio

Congrats! You've successfully built a working RAG for text. Now, let's extend this concept to Video and Audio, and build out some more complex queries. We'll begin by generalizing the above summary creation function to all sorts of modalities.

In [21]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)]) # TODO: Replicate logic similar to the above `create_text_summary` function

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file]) # TODO

    files.append(file) # TODO
    summaries.append(response.text) # TODO

  return files, summaries

Now, we will create a folder with all of our data of different modalities. In particular, the first 5 are audio files, next 3 are text files, and final 6 are video files.

In [23]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/text/images-333.jpg
resources/text/images-023.jpg
resources/text/images-020.jpg
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/Apollo11Intro.mov
Waiting for video file upload...
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...


In [24]:
db = create_chroma_db(all_summaries, "NASA_Apollo11_DB") # TODO: Similar to text_db, create a db but now for all modalities

Again, ensure that the embeddings were generated. Notice that now, we have audio, video, and text data.

In [25]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.05657775700092316, 0.012289989739656448, -0...",Two pilots are talking about the altitude and ...
1,"[0.03358013555407524, -0.011914200149476528, -...",A pilot is discussing flight information with ...
2,"[0.05841799080371857, -0.006945138331502676, -...",This is footage of the Apollo 11 moon landing....
3,"[0.040411096066236496, -0.0201640073210001, -0...",The video shows the launch of the Apollo 11 mi...
4,"[0.04527099058032036, -0.06010293960571289, -0...",Buzz Aldrin descends the Apollo 11 lunar modul...
5,"[0.03521634265780449, -0.02189439907670021, 0....",A video comparing the original footage of Buzz...
6,"[0.03899022936820984, 0.009499304927885532, -0...",Audio of someone discussing a weather radar an...
7,"[0.05688725411891937, -0.014584899879992008, -...",The audio discusses the status of a rocket lau...
8,"[0.036734774708747864, -0.01097866240888834, -...",Astronauts discuss the moon's surface features...
9,"[0.08578625321388245, -0.0016809480730444193, ...",This document outlines assumptions and ground ...


In [26]:
files = get_relevant_files("communication with Mission Control", db)
print(files)

['1', '3', '7']


Can we do more than just return the most relevant file? Yes we can! We can ask Gemini to return a response to the query using the files it thinks are most relevant, provide an answer and tell us what files it used! This is really exciting, and has vast applications in many industries.

In [27]:
def query_rag(query, db):
    files = get_relevant_files(query, db)
    prompt = [all_files[int(f)] for f in files]
    prompt.append("Generate a response to the query using the provided files. Here is the query.")
    prompt.append(query)
    return model.generate_content(prompt).text, [all_file_names[int(f)] for f in files]

In [29]:
for response in query_rag("Explain what happened with the Apollo 11 Mission.", db):
    print(response)

The Apollo 11 mission was the first successful crewed lunar landing. It launched on July 16, 1969 from the Kennedy Space Center in Florida, landing on the moon on July 20, 1969, at 02:56 UTC.  Neil Armstrong and Buzz Aldrin were the first humans to walk on the moon, while Michael Collins remained in lunar orbit.  The mission concluded on July 24, 1969, when the crew splashed down safely in the Pacific Ocean. 
['resources/video/RaisingTheAmericanFlag.mov', 'resources/text/images-333.jpg', 'resources/video/Apollo11PlaqueComparison.mov']


In [30]:
for response in query_rag("What happens at the Translunar Coast in the Mission Description?", db):
    print(response)

The Translunar Coast is the period after TLI, where the spacecraft performs a free lunar return trajectory, the following major events occur prior to LOI: 

* Transposition, docking and LM ejection, including SIVB photography.
* Separation from SIVB and a CSM evasive maneuver.
* SIVB propulsive venting of propellants (slingshot).
* Two series of P23 cislunar navigation sightings, star/earth horizon, consisting of five sets at 06:00 GET and five sets at 24:30 GET.
* Four midcourse corrections which take place at TLI + 9, TLI + 24, LOI - 22 and LOI - 5 hours with ΔV nominally zero. 

['resources/audio/Apollo11OnboardAudioHighlightClip3.mp3', 'resources/video/RaisingTheAmericanFlag.mov', 'resources/video/Apollo11MoonwalkMontage.mov']


Congrats! You've built a full end to end multimodal RAG with just a few tools. We hope you enjoyed following along in this notebook and learned a lot on the way.